In [47]:
import numpy as np
import os
from os.path import expanduser
home_dir = expanduser("~")
module_path = home_dir + '/modules/'
import sys
sys.path.append(module_path)
import pandas as pd
from sklearn.neighbors import BallTree
import math
from itertools import permutations
np.random.seed(999)

In [44]:
#galfile = pd.read_hdf('/scratch/data/galcats/P200/galaxies.Z01.h5')
galfile = pd.read_hdf('test_galcat.h5')
galaxies = galfile.as_matrix()
gal_header = galfile.keys().tolist()
print(gal_header)
print(np.shape(galaxies))

['X_pos', 'Y_pos', 'Z_pos', 'X_vel', 'Y_vel', 'Z_vel', 'Halo_mass', 'Stellar_mass', 'SFR', 'Intra_cluster_mass', 'Halo_mass_peak', 'Stellar_mass_obs', 'SFR_obs', 'Halo_radius', 'Concentration', 'Halo_spin', 'Scale_peak_mass', 'Scale_half_mass', 'Scale_last_MajM', 'Type']
(10000, 20)


In [45]:
data_dict = {'X_pos': 0, 'Y_pos': 1, 'Z_pos': 2, 'X_vel': 3, 'Y_vel': 4, 'Z_vel': 5, 'Halo_mass': 6, 
             'Stellar_mass': 7, 'SFR': 8, 'Intra_cluster_mass': 9, 'Halo_mass_peak': 10, 'Stellar_mass_obs': 11, 
             'SFR_obs': 12, 'Halo_radius': 13, 'Concentration': 14, 'Halo_spin': 15, 'Scale_peak_mass': 16, 
             'Scale_half_mass': 17, 'Scale_last_MajM': 18, 'Type': 19}

In [46]:
nr_neighbours_wanted = 4
nr_dims = 3
halo_masses = galaxies[:, 6]
borders_list = [[0, 200], [0, 200], [0, 200]] # [[x1min, x1max], ... , [xNmin, xNmax]]

### Make sure that the edges are what they should be
print(np.min(galaxies[:, 0]), np.max(galaxies[:, 0]))
print(np.min(galaxies[:, 1]), np.max(galaxies[:, 1]))
print(np.min(galaxies[:, 2]), np.max(galaxies[:, 2]))

0.029570606 199.99576
0.005459643 199.99397
0.024612661 199.9723


In [6]:
### Start of func
coordinates = galaxies[:, :nr_dims].tolist()
if not isinstance(halo_masses, list):
    masses = halo_masses.tolist()
else:
    masses = halo_masses
nr_points = len(coordinates)
volume = 1
box_sides = [] # contains the length of the box in all dimensions
for borders in borders_list:
    volume *= (borders[1] - borders[0])
    box_sides.append(borders[1] - borders[0])
vol_per_point = volume / np.size(coordinates, 0)
dist_between_points = np.power(vol_per_point, 1/3)
buffer_dist = dist_between_points * nr_neighbours_wanted
#print(borders_list)
#print('%.1e' % (volume))
#print('%.1e' % (vol_per_point))
#print(dist_between_points)
#print(buffer_dist)
partitions_list = []

for orig_borders in borders_list:
    if (2*buffer_dist < orig_borders[1] - orig_borders[0]):
        partitions_list.append([orig_borders[0] + buffer_dist, orig_borders[1] - buffer_dist])
    else:
        partitions_list.append([(orig_borders[1] - orig_borders[0])/2 + orig_borders[0],
                       (orig_borders[1] - orig_borders[0])/2 + orig_borders[0]])

print(borders_list)
print('partitions:' ,partitions_list)

inner_volume = 1
for partitions in partitions_list:
    inner_volume *= (partitions[1] - partitions[0])

inner_points_indices = []
outer_points_indices = []

for i in range(nr_points):
    outside_array = np.zeros(nr_dims) # +1: below lower bound, -1: above upper bound
    for i_dim, partitions in enumerate(partitions_list):
        if coordinates[i][i_dim] < partitions[0]: 
            outside_array[i_dim] = 1 
        elif coordinates[i][i_dim] > partitions[1]:
            outside_array[i_dim] = -1
            
    
    if not np.any(outside_array): # check if point is inside inner box
        
        inner_points_indices.append(i)

    else:
        
        dims_crossed = np.count_nonzero(outside_array)
        
        if dims_crossed == 1:
            outer_points_indices.append(i)
            masses.append(masses[i])

            coordinates.append(coordinates[i])
            coordinates[-1] += outside_array*box_sides
        else:
            'TODO: Make use of the permutations function and loop over 
            'the permutations of 2dims crossed and the take the 3dim
            
            crossed_dims_indices = np.nonzero(dims_crossed)
            
            # add across one dim
            for i_dim in crossed_dims_indices:
                outer_points_indices.append(i)
                masses.append(masses[i])

                addition_array = np.zeros(nr_dims)
                addition_array[i_dim] = outside_array[i_dim]
                coordinates.append(coordinates[i])
                coordinates[-1] += addition_array*box_sides
                
            # add across both dims
            outer_points_indices.append(i)
            masses.append(masses[i])
            
            coordinates.append(coordinates[i])
            coordinates[-1] += outside_array*box_sides
            
        
print(len(inner_points_indices), len(outer_points_indices))
print(inner_volume/volume, len(inner_points_indices)/nr_points)

[[0, 200], [0, 200], [0, 200]]
partitions: [[37.13271066890223, 162.86728933109777], [37.13271066890223, 162.86728933109777], [37.13271066890223, 162.86728933109777]]
2136 7864
0.2484701404190547 0.2136


In [55]:
a = [1,2,3]
b = list(permutations(a))
print(b[0][0])

1


In [7]:
print(len(inner_points_indices), len(outer_points_indices))
print(len(coordinates))

2136 7864
17864


In [33]:
tree = BallTree(coordinates, leaf_size=2)
neigh_densities = []
for i_point in range(nr_points):
    dist, ind = tree.query([coordinates[i_point]], k=nr_neighbours_wanted)
#    print(type(ind[0]))
#    print(dist, indices)
    sphere_radius = np.max(dist)
#    print(sphere_radius)
    sphere_volume = (4/3) * math.pi * sphere_radius**3
    neigh_masses = [masses[i] for i in ind[0]]
    density =  sum(neigh_masses) / sphere_volume
    neigh_densities.append(density)

# Make a new pd dataset and save it

In [35]:
pd_dataframe = pd.DataFrame(data=galaxies, columns=gal_header)
pd_dataframe['Environmental_density'] = neigh_densities

In [36]:
pd_dataframe.to_hdf('test_galcat_w_densities.h5', 'w')

# Try loading the newly created galaxy catalogue

In [38]:
galfile = pd.read_hdf('test_galcat_w_densities.h5')
galaxies = galfile.as_matrix()
gal_header = galfile.keys().tolist()
print(gal_header)
print(np.shape(galaxies))

['X_pos', 'Y_pos', 'Z_pos', 'X_vel', 'Y_vel', 'Z_vel', 'Halo_mass', 'Stellar_mass', 'SFR', 'Intra_cluster_mass', 'Halo_mass_peak', 'Stellar_mass_obs', 'SFR_obs', 'Halo_radius', 'Concentration', 'Halo_spin', 'Scale_peak_mass', 'Scale_half_mass', 'Scale_last_MajM', 'Type', 'Environmental_density']
(10000, 21)


In [42]:
print(np.mean(galaxies[:,20]))
print(np.sum(galaxies[:,6]))

64.79929873109062
105017.9585237503
